In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from nltk import word_tokenize,sent_tokenize
import nltk
import re
from sklearn.model_selection import train_test_split
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
!pip install transformers
!pip install Keras-Preprocessing

!git lfs install
!git clone https://huggingface.co/ai-forever/sbert_large_nlu_ru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'sbert_large_nlu_ru'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 40 (delta 6), reused 0 (delta 0), pack-reused 28
Unpacking objects: 100% (40/40), 604.71 KiB | 2.65 MiB/s, done.
Filtering content: 100% (2/2), 3.18 GiB | 92.01 MiB/s, done.


In [3]:
test_df = pd.read_json("/content/drive/MyDrive/NLP_2023_mag/Contur/nlp_test_task_2023/nlp_test_task_2023/dataset/test.json")

In [4]:
device = "cuda"

In [5]:
from transformers import BertConfig,BertForQuestionAnswering
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast(vocab_file = "/content/sbert_large_nlu_ru/vocab.txt",do_lower_case=False)

config = BertConfig.from_pretrained('/content/sbert_large_nlu_ru/config.json')

state=torch.load("/content/drive/MyDrive/NLP_2023_mag/Contur/nlp_test_task_2023/nlp_test_task_2023/Models/Bert_QA_1.pt")
model = BertForQuestionAnswering.from_pretrained(pretrained_model_name_or_path=None, state_dict=state['model'], config=config)
model = model.to(device)

In [6]:
def tokenize_sen(json_data,t = "Train"):
    text = json_data['text']
    encoding = tokenizer(text,add_special_tokens=False)[0]

    if t == "Train":
        # Извекаем начало, конец и лэйбл
        start_sen = json_data['entities'][0]['start']
        end_sen = json_data['entities'][0]['end']
        label = json_data['entities'][0]['label']
        if end_sen==0:
            return encoding, 0, 0, label
        else:
            # Извлекаем номера токенов, которые соответствует start_sen и end_sen-1
            start_token = encoding.char_to_token(start_sen)
            end_token = encoding.char_to_token(end_sen-1)

            if start_token!=None and end_token!=None:
                return encoding, start_token, end_token, label
            else:
                return None,None, None,None
    elif t == "Test":
        # На тесте извлекать start и end не нужно
        label = json_data['entities'][0]['label']
        return encoding, label

In [7]:
def tokenize_sens(json_datas,t = "Train"):
    enc_st_et = []
    if t == "Train":
        for json_data in json_datas:
            d = {}
            # Токенизируем текст и получем его encoding, start, end и label
            e, st, et, lab = tokenize_sen(json_data,t)
            if st!=None and e!=None and et!=None:
                d["encoding"] = e
                d["start_token"] = st
                d["end_token"] = et
                d["label"] = lab
                enc_st_et.append(d)
            
                
    elif t == "Test":
        for json_data in json_datas:
            d = {}
            e,lab = tokenize_sen(json_data,t)
            d["encoding"] = e
            d["label"] = lab
            enc_st_et.append(d)
        
    return enc_st_et

In [8]:
# Инициализируем индексы cls, pad и sep
CLS_ID = tokenizer.cls_token_id
PAD_ID = tokenizer.pad_token_id
SEP_ID  = tokenizer.sep_token_id

def cut_long_sequence(data_dict,t = "Train"):
    # Токенизируем запрос
    enc = tokenizer(data_dict['label'],add_special_tokens=False)[0]
    len_query = len(enc.ids)
    # Формируем структуру данных под Train, Test
    if t == "Train":
        sequences = {
            "input_ids":[],
            "attention_mask":[],
            "start_token":[0],
            "end_token":[0],
            "segment_ids":[],
            "shift":[]
        }
    elif t == "Test":
        sequences = {
            "input_ids":[],
            "attention_mask":[],
            "segment_ids":[],
            "shift":[]
        }
    
    # Получаем ids и attention_mask
    input_ids = data_dict['encoding'].ids
    att_ids = data_dict['encoding'].attention_mask
    # Если len(input_ids)>=510-len_query, то обрезаем наши айди текста и attention_mask к нему
    if len(input_ids)<510-len_query:
        pass
    else:
        input_ids = input_ids[0:(510-len_query)]
        att_ids = att_ids[0:(510-len_query)]
    
    # Длина padding
    pad_len = 510 - len(input_ids) - len_query

    # Формируем segment_ids, 0 - вопрос, 1 - текст
    num_seg_a = len([CLS_ID]+enc.ids+[SEP_ID])
    num_seg_b = len(input_ids + [PAD_ID] * pad_len)
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # Формируем input_ids и attention_mask нужной длины
    input_ids = [CLS_ID]+enc.ids+[SEP_ID]+input_ids + [PAD_ID] * pad_len
    attention_mask = [1] +enc.attention_mask+[1]+att_ids +[PAD_ID] * pad_len

    #Инициализируем структуру данных
    sequences['input_ids'] = input_ids
    sequences['attention_mask'] = attention_mask
    sequences['segment_ids'] = segment_ids

    if t == "Train":
        sequences['start_token'] = [data_dict['start_token']+len([CLS_ID]+enc.ids+[SEP_ID])]
        sequences['end_token'] = [data_dict['end_token']+len([CLS_ID]+enc.ids+[SEP_ID])]
    sequences['shift'] = [len([CLS_ID]+enc.ids+[SEP_ID])]
    return sequences, data_dict['encoding']

In [9]:
def preprosses(data,tp = "Train"):
    encodings = []
    if tp == "Train":
        sequences = {
            "input_ids":[],
            "attention_mask":[],
            "start_token":[],
            "end_token":[],
            "segment_ids":[],
            "shift":[]
        }
        cols = ["input_ids", "attention_mask", "start_token", "end_token", "segment_ids","shift"]
    elif tp == "Test":
        sequences = {
            "input_ids":[],
            "attention_mask":[],
            "segment_ids":[],
            "shift":[]
        }
        cols = ["input_ids", "attention_mask", "segment_ids","shift"]

    # Токенизируем данные
    enc_st_et = tokenize_sens(data,tp)

    for i in range(len(enc_st_et)):
        # Формируем данные под QA модель, обрезаем длинные последовательности
        sequence,enc = cut_long_sequence(enc_st_et[i],tp)
        encodings.append(enc)
        for t in cols:
            sequences[t].append(sequence[t])
    
    for t in cols:
        sequences[t] = np.array(sequences[t])
    return sequences,encodings

In [10]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self,data_input, data_att, segment_ids,shift):
        self.data_input = data_input
        self.data_att = data_att
        self.segment_ids = segment_ids
        self.shift = shift
    def __len__(self):
        return len(self.data_input)
    
    def __getitem__(self,idx):
        return torch.from_numpy(self.data_input[idx]),torch.from_numpy(self.data_att[idx]),torch.from_numpy(self.segment_ids[idx]),torch.from_numpy(self.shift[idx])

In [11]:
def predict_answer(input_label, input_text):
    d = {'label': input_label,'text': input_text}
    test_df = pd.DataFrame(data=d)

    data_test = []
    for i in range(len(test_df)):
        data = {}
        data['text'] = test_df['text'][i]
        data['entities'] = []
        lab = test_df['label'][i]

        data['entities'].append({"label":lab})
        data_test.append(data)

        sequences_test,encodings_test = preprosses(data_test,tp="Test")

    test_segment_ids = []
    test_attention_mask = []
    test_input_ids = []
    test_shift = []
    for i in range(len(sequences_test['input_ids'])):
        test_attention_mask.append(sequences_test['attention_mask'][i])
        test_input_ids.append(sequences_test['input_ids'][i])
        test_segment_ids.append(sequences_test['segment_ids'][i])
        test_shift.append(sequences_test['shift'][i])

    test_segment_ids = np.array(test_segment_ids)
    test_attention_mask = np.array(test_attention_mask)
    test_input_ids = np.array(test_input_ids)
    test_shift = np.array(test_shift)

    test_dataset = TestDataset(test_input_ids, test_attention_mask, test_segment_ids,test_shift)
    test_dataloader = torch.utils.data.DataLoader(test_dataset,shuffle=False,batch_size=4,num_workers=2,pin_memory=True)


    model.eval()#переводим сеть в состояние тестирования
    eval_loss=0
    predicted_start = []
    predicted_end = []
    shifts = []

    with torch.no_grad():#не вычисляем параметры
        for batch_inputs, batch_att, batch_segment, shift in test_dataloader:

            batch_inputs = batch_inputs.to(device) # переводим на cudu
            batch_att = batch_att.to(device)
            batch_segment = batch_segment.to(device)

            outputs = model(
                input_ids = batch_inputs, attention_mask = batch_att,
                token_type_ids = batch_segment
                )
            
            predicted_start.extend(torch.argmax(outputs.start_logits,axis=-1).detach().cpu().numpy())
            predicted_end.extend(torch.argmax(outputs.end_logits,axis=-1).detach().cpu().numpy())
            shifts.extend(shift.numpy())

    predicted_start_r = []
    predicted_end_r = []

    for i in range(len(predicted_end)):

        if predicted_start[i] - shifts[i][0] < 0 or predicted_end[i]-shifts[i][0] < 0:
            predicted_start_r.append(0)
            predicted_end_r.append(0)
        else:
            if encodings_test[i].token_to_chars(predicted_start[i] - shifts[i][0]) != None and encodings_test[i].token_to_chars(predicted_end[i]-shifts[i][0])!= None:
                if predicted_end[i] - shifts[i][0] == 0:
                    predicted_start_r.append(0)
                    predicted_end_r.append(0)
                else: 
                    start,_ = encodings_test[i].token_to_chars(predicted_start[i] - shifts[i][0])
                    _,end = encodings_test[i].token_to_chars(predicted_end[i] - shifts[i][0])
                    predicted_start_r.append(start)
                    predicted_end_r.append(end)
            else:
                predicted_start_r.append(0)
                predicted_end_r.append(0)

    for i in range(len(predicted_start_r)):
        if predicted_start_r[i]>=predicted_end_r[i]:
            predicted_start_r[i]=0
            predicted_end_r[i]=0

    test_df["answer_start"] = 0
    test_df["answer_end"] = 0
    test_df["answer_text"] = 0

    # Заполняем таблица answer_start, answer_end, answer_text
    for i in range(len(predicted_start_r)):
        test_df['answer_start'][i]=predicted_start_r[i]
        test_df['answer_end'][i]=predicted_end_r[i]
        test_df['answer_text'][i]=data_test[i]['text'][predicted_start_r[i]:predicted_end_r[i]]

    return test_df

In [ ]:
input_text = test_df['text'][0:2]
input_label = test_df['label'][0:2]
predict_answer(input_label, input_text)